In [1]:
import warnings

import numpy as np
import pandas as pd
import pingouin as pg
import statsmodels.api as sm
from scipy.stats import shapiro, levene
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('source/FReDA4.csv')
df2 = pd.read_csv('source/FReDA3.csv')
df["Group4"] = None

In [3]:
# satisfied = df[df["Group3"] == "Couple Satisfaction"].copy()
# mixed = df[df["Group3"] == "Couple Mixed"].copy()
# deprived = df[df["Group3"] == "Couple Deprivation"].copy()
# saturated = df[df["Group3"] == "Couple Saturation"].copy()
# deprived_one =  df[df["Group2"] == "One-sided Deprivation"].copy()
# deprived_both = df[df["Group1"] == "SubGroup3"].copy()
# deprived_me = df[df["Group1"] == "SubGroup2"].copy()
# deprived_partner = df[df["Group1"] == "SubGroup7"].copy()
# saturated_one =  df[df["Group2"] == "One-sided Saturation"].copy()
# saturated_both = df[df["Group1"] == "SubGroup6"].copy()
# saturated_me = df[df["Group1"] == "SubGroup5"].copy()
# saturated_partner = df[df["Group1"] == "SubGroup8"].copy()

In [4]:
# Satisfied
df.loc[df['Group3'] == 'Couple Satisfaction', 'Group4'] = 'Couple Satisfaction'

# Deprived groups
# df.loc[df['Group3'] == 'Couple Deprivation', 'Group4'] = 'Couple Deprivation'
df.loc[df['Group2'] == 'One-sided Deprivation', 'Group4'] = 'Deprived_One'
df.loc[df['Group1'] == 'SubGroup3', 'Group4'] = 'Deprived_Both'
#
# Saturated groups
# df.loc[df['Group3'] == 'Couple Saturation', 'Group4'] = 'Couple Saturation'
df.loc[df['Group2'] == 'One-sided Saturation', 'Group4'] = 'Saturated_One'
df.loc[df['Group1'] == 'SubGroup6', 'Group4'] = 'Saturated_Both'

df.loc[df['Group3'] == 'Couple Mixed', 'Group4'] = 'Couple Mixed'


In [5]:
unique_values = df['Group4'].value_counts()
print(unique_values)

Group4
Deprived_One           4940
Couple Satisfaction    3842
Deprived_Both          3402
Saturated_One           666
Couple Mixed            660
Saturated_Both           90
Name: count, dtype: int64


In [6]:
df = df.rename(columns={
    'Self-esteem': 'Self_esteem',
    'Life Satisfaction': 'Life_Satisfaction',
    'Communication Quality': 'Communication_Quality',
    'Relationship Satisfaction': 'Relationship_Satisfaction',
    'Conflict Management': 'Conflict_Management'
})

In [7]:
traits = [
    'Neuroticism',
    'Extraversion',
    'Openness',
    'Agreeableness',
    'Conscientiousness',

    'Depressiveness',
    'Loneliness',
    'Self_esteem',
    'Life_Satisfaction',
    'Health',

    'Relationship_Satisfaction',
    'Communication_Quality',
    'Conflict_Management'
]

In [8]:
df = df.dropna(subset=traits).copy()

In [9]:
# -----------------------
# Assumption checks
# -----------------------
for trait in traits:
    print(f"\n--- {trait} ---")

    # Shapiro normality per group
    for group in df['Group4'].unique():
        data = df[df['Group4'] == group][trait]
        if len(data) >= 3:  # Shapiro requires >=3
            stat, p = shapiro(data)
            # print(f"{group} Shapiro p={p:.3f}")

    # Levene test for homogeneity of variance
    groups_data = [df[df['Group4'] == g][trait] for g in df['Group4'].unique()]
    stat, p = levene(*groups_data)
    print(f"Levene test p={p:.3f}")


--- Neuroticism ---
Levene test p=0.021

--- Extraversion ---
Levene test p=0.642

--- Openness ---
Levene test p=0.286

--- Agreeableness ---
Levene test p=0.002

--- Conscientiousness ---
Levene test p=0.506

--- Depressiveness ---
Levene test p=0.086

--- Loneliness ---
Levene test p=0.000

--- Self_esteem ---
Levene test p=0.000

--- Life_Satisfaction ---
Levene test p=0.004

--- Health ---
Levene test p=0.000

--- Relationship_Satisfaction ---
Levene test p=0.000

--- Communication_Quality ---
Levene test p=0.000

--- Conflict_Management ---
Levene test p=0.000


In [10]:
for trait in traits:
    # print(f"\n=== ANOVA for {trait} ===")

    welch_anova = pg.welch_anova(dv=trait, between='Group4', data=df)
    # print(welch_anova)
    f_val = welch_anova.at[0, 'F']
    df_between = int(welch_anova.at[0, 'ddof1'])
    df_within = welch_anova.at[0, 'ddof2']  # Keep as float for Welch
    p_val = welch_anova.at[0, 'p-unc']
    eta_sq = welch_anova.at[0, 'np2']

    report = f"F({df_between}, {df_within:.2f}) = {f_val:.2f}; p = {p_val:.2E}; partial η2 = {eta_sq:.3f}"
    print(report)

F(5, 678.20) = 32.13; p = 2.00E-29; partial η2 = 0.013
F(5, 678.70) = 6.29; p = 1.02E-05; partial η2 = 0.003
F(5, 682.40) = 8.54; p = 7.39E-08; partial η2 = 0.003
F(5, 677.24) = 10.87; p = 4.52E-10; partial η2 = 0.005
F(5, 677.74) = 2.77; p = 1.75E-02; partial η2 = 0.001
F(5, 679.67) = 32.63; p = 7.32E-30; partial η2 = 0.013
F(5, 675.66) = 83.13; p = 5.09E-68; partial η2 = 0.033
F(5, 676.68) = 48.14; p = 1.23E-42; partial η2 = 0.019
F(5, 679.15) = 59.46; p = 2.27E-51; partial η2 = 0.024
F(5, 677.84) = 34.34; p = 2.53E-31; partial η2 = 0.014
F(5, 681.03) = 258.11; p = 1.60E-154; partial η2 = 0.087
F(5, 678.48) = 244.15; p = 5.39E-149; partial η2 = 0.089
F(5, 679.89) = 100.52; p = 2.59E-79; partial η2 = 0.039


In [11]:
# # -----------------------
# # ANOVA + Effect size
# # -----------------------
# for trait in traits:
#     print(f"\n=== ANOVA for {trait} ===")
#
#     # OLS model
#     model = ols(f'{trait} ~ C(Group4)', data=df).fit()
#
#     # Standard ANOVA table
#     anova_table = sm.stats.anova_lm(model, typ=2)
#     print(anova_table)
#
#     # Eta squared from pingouin
#     eta2 = pg.anova(dv=trait, between='Group4', data=df,  effsize='n2', detailed=True)
#     print(f"Eta squared = {eta2.loc[0, 'n2']:.3f}")

In [12]:
print(df["Group4"].nunique())

6


In [13]:
# -----------------------
# Post-hoc tests
# -----------------------

traits = [
    'Neuroticism',
    'Extraversion',
    'Openness',
    'Agreeableness',
    'Conscientiousness',

    'Depressiveness',
    'Loneliness',
    'Self_esteem',
    'Life_Satisfaction',
    'Health',

    'Relationship_Satisfaction',
    'Communication_Quality',
    'Conflict_Management'
]

from pingouin import pairwise_tukey
all_results = []
for trait in traits:
    print(f"\n--- Post-hoc comparisons for {trait} ---")

    # Tukey HSD
    tukey = pairwise_tukey(
        dv=trait,
        between='Group4',
        data=df,
        effsize='cohen'
    )
    filter_turkey = tukey.loc[[5,6,7,8]].reset_index(drop=True)
    filter_turkey['Trait'] = trait
    all_results.append(filter_turkey)



--- Post-hoc comparisons for Neuroticism ---

--- Post-hoc comparisons for Extraversion ---

--- Post-hoc comparisons for Openness ---

--- Post-hoc comparisons for Agreeableness ---

--- Post-hoc comparisons for Conscientiousness ---

--- Post-hoc comparisons for Depressiveness ---

--- Post-hoc comparisons for Loneliness ---

--- Post-hoc comparisons for Self_esteem ---

--- Post-hoc comparisons for Life_Satisfaction ---

--- Post-hoc comparisons for Health ---

--- Post-hoc comparisons for Relationship_Satisfaction ---

--- Post-hoc comparisons for Communication_Quality ---

--- Post-hoc comparisons for Conflict_Management ---


In [14]:
final_turkey = pd.concat(all_results, ignore_index=True)